In [47]:
# Beautiful soup, requests and pandas (pandas used here for csv wrangling)
# Beautiful soup, requests and pandas (pandas used here for csv wrangling)
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re # needed for regular expression to strip out grammar
import json

# Selenium imports for automating web browsing
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
import time

ft_folder = 'N:\\SRDStaff\\GPS\\SPI\\Insight\\Personal Workspace\\Frank\\Webscraping\\'
print('Imports loaded')

folder_target_str = "C:\\My Computer\\"

Imports loaded


# Huddersfield
## The home of the infinite scroll...
- Had to find the API where it's requesting the data and get from there

In [ ]:
# find the url for the json file
# Go to develpor mode - open Network and look at Fetch/XHR
url = "https://courses.hud.ac.uk/json/2025-26/sort:title"
response = requests.get(url)
if response.status_code == 200:
    courses = response.json()
    #for course in courses:
        #print(course)  # Adjust according to the structure of the response
else:
    print(f"Error: {response.status_code}")

In [ ]:
# This returns a JSON file that acts like a dictionary
year=courses['courses'][0]['year']
study_mode=courses['courses'][0]['study_mode']
study_level=courses['courses'][0]['study_level']
urltitle=courses['courses'][0]['urltitle']
'https://courses.hud.ac.uk/'+year +'/'+study_mode+'/'+study_level+'/'+urltitle

In [ ]:
course_list=[]

for i in range(len(courses['courses'])):
    year=courses['courses'][i]['year']
    study_mode=courses['courses'][i]['study_mode']
    study_level=courses['courses'][i]['study_level']
    urltitle=courses['courses'][i]['urltitle']
    course_title = courses['courses'][i]['title']
    href='https://courses.hud.ac.uk/'+year +'/'+study_mode+'/'+study_level+'/'+urltitle
    course_list.append([course_title,href])
print('course list created')

## University of Lincoln
- Selenium used to get course hrefs

In [ ]:
# Initialise Chrome Driver
driver = webdriver.Chrome()
url='https://www.lincoln.ac.uk/search/?searchStudioQuery=&facetId=courses&isGrid=false&facets=fq%3DsectionType_s%3A%22courses%22&orderBy=&start=0'
driver.get(url)
driver.implicitly_wait(20)
wait = WebDriverWait(driver, 10)  # Timeout after 10 seconds
div_class = 'courses' # find a within this
course_list = []
# next link
next_class = "page-link-searchstudio-js" # Text contains 'Next'
search_results_wrapper = 'search-results-wrapper'
# test while loop
test_loop = False
if test_loop:
    total_courses=41
else:
    total_courses=411
# we need to click the next button for a set amount of iterations:
i = 1

while i<total_courses+10:
    time.sleep(10)
    div_elements = driver.find_elements(By.CLASS_NAME, div_class)
    for div in div_elements:
        href = div.find_element(By.TAG_NAME, "a")
        course_list.append([href.text,href.get_attribute("href")])
    # find the next button and click it
    link_elements = driver.find_elements(By.CLASS_NAME, next_class)
    for link in link_elements:
        if 'Next' in link.text:
            link.click()
    i+=10
print('Courses found')

## Nottingham Trent
- A beautiful soup based webscrape

In [14]:
def extract_hrefs(url,class_target="courseAToZ__list",print_outputs=False):
    """
    This function is intended to return a list of all hrefs from a particular webpage within a list
    Inputs:
    url - The target url, str
    class_target - the class string which contains the list, str
    Output:
    output_list - a list [course_name, course_url]
    """
    response = requests.get(url)
    html_content = response.text
    # Parse the HTML with Beautiful Soup
    soup = BeautifulSoup(html_content, "html.parser")
    # Locate the <ul> element with class "courseAToZ__list"
    ul_element = soup.find("ul", {"class": class_target})
    # create output_list
    output_list = []
    # Ensure the <ul> exists
    if ul_element:
        # Find all <li> elements within the <ul>
        list_items = ul_element.find_all("li")
        # Extract and print the text content of each <li>
        for li in list_items:
            temp_list = []
            # get text from list
            course_name = li.get_text(strip=True)
            temp_list.append(course_name)
            if print_outputs:
                print(course_name)
            a_tag = li.find("a")  # Find the <a> tag inside the <li>
            if a_tag:
                href = a_tag.get("href")  # Get the href attribute
                temp_list.append(href)
                if print_outputs:
                    print("Link:", href)
            output_list.append(temp_list)
    else:
        print("No <ul> with class 'courseAToZ__list' found.")
    return output_list

In [11]:
root_url = "https://www.ntu.ac.uk/study-and-courses/undergraduate/course-a-z?result_1908621_result_page="
alphabet = list('ABCDEFGHIJKLMNOPQRSTUVWXYZ')

In [17]:
# Create list of each course that we're going to save
final_list = []
for a in alphabet:
    # create our url from our root_url
    url = root_url+a
    print(url)
    # Call extract_hrefs on our url and add it to our final_list
    final_list += extract_hrefs(url,class_target="courseAToZ__list")

https://www.ntu.ac.uk/study-and-courses/undergraduate/course-a-z?result_1908621_result_page=A
https://www.ntu.ac.uk/study-and-courses/undergraduate/course-a-z?result_1908621_result_page=B
https://www.ntu.ac.uk/study-and-courses/undergraduate/course-a-z?result_1908621_result_page=C
https://www.ntu.ac.uk/study-and-courses/undergraduate/course-a-z?result_1908621_result_page=D
https://www.ntu.ac.uk/study-and-courses/undergraduate/course-a-z?result_1908621_result_page=E
https://www.ntu.ac.uk/study-and-courses/undergraduate/course-a-z?result_1908621_result_page=F
https://www.ntu.ac.uk/study-and-courses/undergraduate/course-a-z?result_1908621_result_page=G
https://www.ntu.ac.uk/study-and-courses/undergraduate/course-a-z?result_1908621_result_page=H
https://www.ntu.ac.uk/study-and-courses/undergraduate/course-a-z?result_1908621_result_page=I
https://www.ntu.ac.uk/study-and-courses/undergraduate/course-a-z?result_1908621_result_page=J
https://www.ntu.ac.uk/study-and-courses/undergraduate/course

In [ ]:
df = pd.DataFrame(final_list,columns=['Course_Title','Course_Href'])
df.drop_duplicates(inplace=True)
output_file_name = 'Nott Trent Course List.csv'
df.to_csv(folder_target_str+output_file_name)

In [38]:
def get_href_text(url,class_header_target = 'course-modules__intro',class_div_target = 'course-modules__tab-content'):
    """
    This function get the header elements and module elements of the course
    url - The target url, str
    class_header_target - the class string which contains the header, str
    class_div_target - the class string which contains the module data, str
    Output:
    output_list - a list [header information, module information]
    """
    response = requests.get(url)
    html_content = response.text
    # Parse the HTML with Beautiful Soup
    soup = BeautifulSoup(html_content, "html.parser")
    header_element = soup.find("header", {"class": class_header_target})
    div_element = soup.find("div", {"class": class_div_target})
    #header_text
    if header_element!=None:
        header_text= header_element.get_text(strip=True)
    else:
        header_text='No Value'
    #div_text - contains module data
    if div_element!=None:
        div_text= div_element.get_text(strip=True)
    else:
        div_text='No Value'
    output_list = [header_text,div_text]
    return output_list

# Iterate over dataframe and find all module headers and module info

In [40]:
# new outputlist we're creating
final_list = []
# loop through dataframe
m,n = df.shape # get the shape of the dataframe
for row in range(m):
    temp_list = list(df.iloc[row,:])
    url = df.iloc[row,1]
    temp_list +=get_href_text(url)
    final_list.append(temp_list)
print('Output created')

Output created


# Output to CSV

In [43]:
df = pd.DataFrame(final_list,columns=['Course_Title','Course_Href','Course Header','Module Info'])
df.drop_duplicates(inplace=True)
output_file_name = 'Nott Trent Course List.csv'
df.to_csv(folder_target_str+output_file_name)

# Find only AI related courses

In [48]:
# new outputlist we're creating
final_list = []
# loop through dataframe
m,n = df.shape # get the shape of the dataframe
for row in range(m):
    header_text = df.iloc[row,2].upper()
    header_text=re.sub(r"[^\w\s]", " ", header_text) # strip out grammar
    module_text = df.iloc[row,3].upper()
    module_text=re.sub(r"[^\w\s]", " ", module_text)
    # test for AI or artificial intelligence
    if ('AI ' in header_text) or ('AI ' in module_text) or ('ARTIFICIAL INT' in header_text) or ('ARTIFICIAL INT' in module_text):
        final_list.append(list(df.iloc[row,:]))

In [49]:
ai_df = pd.DataFrame(final_list,columns=['Course_Title','Course_Href','Course Header','Module Info'])
output_file_name = 'Nott Trent AI Course List.csv'
ai_df.to_csv(folder_target_str+output_file_name)

In [ ]:
# get all post-grad areas
div_class = "component__container"
